In [11]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
import xgboost as xgb
import warnings
import random
import joblib
warnings.filterwarnings("ignore")

# ── Global seed — change this one value to reproduce any run ────────────────
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

## 1. Load Data

In [12]:
df_train = pd.read_csv("/kaggle/input/datasets/yassinechelly4/dataoverflow2/train.csv")
df_test  = pd.read_csv("/kaggle/input/datasets/yassinechelly4/dataoverflow2/test.csv")

TARGET   = "Purchased_Coverage_Bundle"
test_ids = df_test["User_ID"].copy()

# ── Columns to drop
# Policy_Cancelled_Post_Purchase: kept — has small but real signal (corr=0.03)
# Broker_ID: dropped (8331 nulls) BUT we extract a binary flag first
# Employer_ID: 57396/60868 null — nearly all missing, drop
# Policy_Start_Week / Day: dropped — cyclically encoded or low signal
DROP_COLS = ["User_ID", "Employer_ID", "Policy_Start_Week", "Policy_Start_Day"]

df_train = df_train.drop(columns=DROP_COLS, errors="ignore")
df_test  = df_test.drop(columns=DROP_COLS, errors="ignore")
print(f"Train shape: {df_train.shape} | Test shape: {df_test.shape}")

Train shape: (60868, 25) | Test shape: (15218, 24)


## 2. Feature Engineering

All features engineered from domain knowledge + correlation analysis. Applied identically to train and test before any encoding.

In [13]:
def engineer_features(df, is_train=True):
    df = df.copy()

    # ── Binary flags (extracted BEFORE imputation so missingness is captured)
    df["Has_Broker"]   = df["Broker_ID"].notna().astype(int)
    df["Has_Employer"] = (df.get("Employer_ID", pd.Series(np.nan, index=df.index)).notna()).astype(int)

    # ── Impute Broker_ID with -1 (unknown) then keep as numeric signal
    df["Broker_ID"] = df["Broker_ID"].fillna(-1)

    # ── Dependent aggregates
    df["Child_Dependents"]    = df["Child_Dependents"].fillna(0)
    df["Total_Dependents"]    = df["Adult_Dependents"] + df["Child_Dependents"] + df["Infant_Dependents"]
    df["Minor_Dependents"]    = df["Child_Dependents"] + df["Infant_Dependents"]

    # ── Income features
    df["Log_Income"]          = np.log1p(df["Estimated_Annual_Income"])
    df["Income_Per_Dependent"]= df["Estimated_Annual_Income"] / (df["Total_Dependents"] + 1)
    df["Income_Bracket"]      = pd.qcut(df["Estimated_Annual_Income"], q=5, labels=False, duplicates="drop").astype(float)

    # ── Policy risk features
    df["Policy_Complexity"]   = df["Vehicles_on_Policy"] + df["Custom_Riders_Requested"] + df["Policy_Amendments_Count"]
    df["Grace_Ratio"]         = df["Grace_Period_Extensions"] / (df["Previous_Policy_Duration_Months"] + 1)
    df["Claims_Rate"]         = df["Previous_Claims_Filed"] / (df["Previous_Policy_Duration_Months"] + 1)
    df["Clean_Ratio"]         = df["Years_Without_Claims"] / (df["Previous_Policy_Duration_Months"] + 1)

    # ── Deductible: this IS an ordered variable (tier 1 > tier 4)
    ded_map = {"Tier_1_High_Ded": 3, "Tier_2_Mid_Ded": 2, "Tier_3_Low_Ded": 1, "Tier_4_Zero_Ded": 0}
    df["Deductible_Ord"] = df["Deductible_Tier"].map(ded_map).fillna(-1)
    df = df.drop(columns=["Deductible_Tier"])

    # ── Cyclical month encoding (preserves Jan-Dec circularity)
    month_map = {"January":1,"February":2,"March":3,"April":4,"May":5,"June":6,
                 "July":7,"August":8,"September":9,"October":10,"November":11,"December":12}
    df["Month_Num"] = df["Policy_Start_Month"].map(month_map).fillna(0)
    df["Month_Sin"] = np.sin(2 * np.pi * df["Month_Num"] / 12)
    df["Month_Cos"] = np.cos(2 * np.pi * df["Month_Num"] / 12)
    df = df.drop(columns=["Policy_Start_Month", "Month_Num"])

    return df

df_train = engineer_features(df_train)
df_test  = engineer_features(df_test)
print(f"Features after engineering: {df_train.shape[1]}")

Features after engineering: 37


## 3. Imputation & Encoding

In [14]:
# ── Identify column types (after engineering)
num_cols = df_train.select_dtypes(include=[np.number]).columns.drop(TARGET, errors="ignore").tolist()
# Remaining categorical cols (Deductible_Tier already ordinal-encoded above)
cat_cols = [c for c in df_train.select_dtypes(include=["object"]).columns
            if c not in [TARGET, "Region_Code"]]

print("Numeric cols :", len(num_cols))
print("Nominal cats :", cat_cols)
print("High-card cat: Region_Code (166 unique) → target encoding")

# ── Numeric imputation with TRAIN medians (applied to both)
medians = df_train[num_cols].median()
df_train[num_cols] = df_train[num_cols].fillna(medians)
df_test[num_cols]  = df_test[num_cols].fillna(medians)

# ── Nominal categoricals: mode imputation then OHE
for col in cat_cols:
    mode_val = df_train[col].mode()[0]
    df_train[col] = df_train[col].fillna(mode_val)
    df_test[col]  = df_test[col].fillna(mode_val)

# NOTE: OrdinalEncoder assigns arbitrary integers to unordered categories.
# For XGBoost, One-Hot Encoding is strictly better for nominal cats
# because OE implies a false ordering (e.g. Urban_Boutique=0 < National_Corporate=1).
df_train = pd.get_dummies(df_train, columns=cat_cols, drop_first=False, dtype=int)
df_test  = pd.get_dummies(df_test,  columns=cat_cols, drop_first=False, dtype=int)

# ── Align columns (test may be missing some OHE columns from rare train values)
df_test = df_test.reindex(columns=df_train.drop(columns=[TARGET]).columns, fill_value=0)

# ── Target encoding for Region_Code (high-cardinality: 166 unique values)
# OrdinalEncoder on 166 unordered countries is very noisy; target encoding
# captures the actual mean outcome per region with Bayesian smoothing.
df_train["Region_Code"] = df_train["Region_Code"].fillna("Unknown")
df_test["Region_Code"]  = df_test["Region_Code"].fillna("Unknown")

global_mean = df_train[TARGET].mean()
smoothing   = 20
region_stats = df_train.groupby("Region_Code")[TARGET].agg(["mean","count"])
region_stats["encoded"] = (
    (region_stats["mean"] * region_stats["count"] + global_mean * smoothing)
    / (region_stats["count"] + smoothing)
)
region_map = region_stats["encoded"].to_dict()
df_train["Region_Enc"] = df_train["Region_Code"].map(region_map).fillna(global_mean)
df_test["Region_Enc"]  = df_test["Region_Code"].map(region_map).fillna(global_mean)
df_train = df_train.drop(columns=["Region_Code"])
df_test  = df_test.drop(columns=["Region_Code"])

print(f"Final feature count: {df_train.shape[1] - 1}")

Numeric cols : 31
Nominal cats : ['Broker_Agency_Type', 'Acquisition_Channel', 'Payment_Schedule', 'Employment_Status']
High-card cat: Region_Code (166 unique) → target encoding
Final feature count: 46


## 4. Prepare X / y

> **Note:** StandardScaler removed. XGBoost is tree-based — it uses threshold comparisons on raw feature values. Scaling has literally zero effect on any split decision and only adds preprocessing complexity and a potential train/test leakage vector.

In [15]:
X = df_train.drop(columns=[TARGET])
y = df_train[TARGET].values.astype(int)
X_test_final = df_test.copy()

le = LabelEncoder()
y_enc = le.fit_transform(y)
NUM_CLASSES = len(le.classes_)
print(f"Classes: {le.classes_}  ({NUM_CLASSES} total)")
print(f"\nClass distribution:\n{pd.Series(y_enc).value_counts().sort_index()}")
feature_names = list(X.columns)


Classes: [0 1 2 3 4 5 6 7 8 9]  (10 total)

Class distribution:
0      823
1     1625
2    36136
3     4831
4    13958
5      479
6      719
7     2286
8        6
9        5
Name: count, dtype: int64


## 5. Train / Validation Split

In [16]:
X_train, X_val, y_train, y_val = train_test_split(
    X.values, y_enc,
    test_size=0.1, random_state=SEED, stratify=y_enc
)
print(f"Train: {X_train.shape} | Val: {X_val.shape}")

Train: (54781, 46) | Val: (6087, 46)


## 6. Balanced Resampling — SMOTETomek (targeted)

**Problem with the original approach:**
The original SMOTE resampled ALL classes up to the majority class size (32,522).
Classes 8 and 9 had only 5–6 real samples — after resampling, **98% of their training data was purely synthetic**, completely hallucinated by KNN interpolation.
This causes the model to memorise synthetic artefacts, not real patterns.

**Fix — two-part strategy:**
1. **Moderate oversampling** of rare classes (capped at 200 samples, not 32,522)
2. **Tomek Links cleaning** removes borderline majority samples near minority boundaries, giving cleaner decision boundaries without inflating data volume.
3. **XGBoost ** handles the remaining imbalance at training time without synthetic data at all.

In [17]:
print("Before resampling:", pd.Series(y_train).value_counts().sort_index().to_dict())

# Count per class — only oversample classes that are severely under-represented
class_counts = pd.Series(y_train).value_counts()
target_count  = 500   # oversample tiny classes to at most 500 (not 32522)

sampling_strategy = {}
for cls, cnt in class_counts.items():
    if cnt < target_count:
        sampling_strategy[cls] = min(target_count, cnt * 50)  # cap at 50x original

print(f"SMOTE sampling targets: {sampling_strategy}")

k_neighbors = max(1, min(5, class_counts.min() - 1))
smote = SMOTE(random_state=SEED, k_neighbors=k_neighbors, sampling_strategy=sampling_strategy)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print("After resampling:", pd.Series(y_train_res).value_counts().sort_index().to_dict())
print(f"Train size: {len(X_train_res):,}  (was {len(X_train):,})")

Before resampling: {0: 741, 1: 1463, 2: 32522, 3: 4348, 4: 12562, 5: 431, 6: 647, 7: 2057, 8: 5, 9: 5}
SMOTE sampling targets: {5: 500, 8: 250, 9: 250}
After resampling: {0: 741, 1: 1463, 2: 32522, 3: 4348, 4: 12562, 5: 500, 6: 647, 7: 2057, 8: 250, 9: 250}
Train size: 55,340  (was 54,781)


## 7. Class Weights for XGBoost

In [18]:
from sklearn.utils.class_weight import compute_sample_weight

# Compute per-sample weights from resampled labels
# This handles remaining imbalance at the loss level — no fake data needed
sample_weights = compute_sample_weight(class_weight="balanced", y=y_train_res)
print(f"Sample weight range: {sample_weights.min():.4f} – {sample_weights.max():.4f}")

Sample weight range: 0.1702 – 22.1360


## 8. Train XGBoost

**Key improvements:**
- : stops training when val loss stops improving (the original had NO early stopping — val loss was still decreasing at tree 299, meaning it was undertrained)
-  passed to fit instead of pure SMOTE inflation
- Removed redundant  (no effect on tree models)
- Added  to track both loss and error

In [20]:
xgb_model = xgb.XGBClassifier(
    objective         = "multi:softprob",
    num_class         = NUM_CLASSES,
    eval_metric       = ["mlogloss", "merror"],
    n_estimators      = 1000,
    max_depth         = 5,
    learning_rate     = 0.1,
    subsample         = 0.8,
    colsample_bytree  = 0.8,
    random_state      = SEED,
    n_jobs            = -1,
    verbosity         = 0,
    early_stopping_rounds = 50,
)
xgb_model.fit(
    X_train_res, y_train_res,
    sample_weight     = sample_weights,
    eval_set          = [(X_val, y_val)],
    verbose           = 50,
)
print(f"\nBest iteration: {xgb_model.best_iteration}")
print(f"Best val mlogloss: {xgb_model.best_score:.5f}")

[0]	validation_0-mlogloss:2.16036	validation_0-merror:0.55890
[50]	validation_0-mlogloss:1.11456	validation_0-merror:0.44932
[100]	validation_0-mlogloss:0.99891	validation_0-merror:0.42057
[150]	validation_0-mlogloss:0.93635	validation_0-merror:0.39494
[200]	validation_0-mlogloss:0.89421	validation_0-merror:0.37539
[250]	validation_0-mlogloss:0.86174	validation_0-merror:0.36028
[300]	validation_0-mlogloss:0.83748	validation_0-merror:0.35058
[350]	validation_0-mlogloss:0.81575	validation_0-merror:0.33843
[400]	validation_0-mlogloss:0.80022	validation_0-merror:0.33087
[450]	validation_0-mlogloss:0.78521	validation_0-merror:0.32282
[500]	validation_0-mlogloss:0.77322	validation_0-merror:0.31493
[550]	validation_0-mlogloss:0.76255	validation_0-merror:0.31017
[600]	validation_0-mlogloss:0.75444	validation_0-merror:0.30754
[650]	validation_0-mlogloss:0.74655	validation_0-merror:0.30409
[700]	validation_0-mlogloss:0.73851	validation_0-merror:0.29883
[750]	validation_0-mlogloss:0.73319	validat

## 9. Evaluate

In [21]:
y_pred = xgb_model.predict(X_val)

print("Accuracy :", round(accuracy_score(y_val, y_pred), 4))
print("Macro F1  :", round(f1_score(y_val, y_pred, average="macro",    zero_division=0), 4))
print("Weighted F1:",round(f1_score(y_val, y_pred, average="weighted", zero_division=0), 4))
print()
print(classification_report(y_val, y_pred, zero_division=0))

Accuracy : 0.7038
Macro F1  : 0.5508
Weighted F1: 0.7204

              precision    recall  f1-score   support

           0       0.42      0.76      0.54        82
           1       0.61      0.69      0.65       162
           2       0.91      0.74      0.82      3614
           3       0.36      0.66      0.46       483
           4       0.59      0.62      0.61      1396
           5       0.68      0.71      0.69        48
           6       0.54      0.61      0.58        72
           7       0.53      0.72      0.61       229
           8       0.00      0.00      0.00         1

    accuracy                           0.70      6087
   macro avg       0.52      0.61      0.55      6087
weighted avg       0.76      0.70      0.72      6087



## 10. Feature Importance (Top 20)

In [22]:
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

importance = pd.Series(xgb_model.feature_importances_, index=feature_names).sort_values(ascending=False)
print("Top 20 features:")
print(importance.head(20).to_string())

fig, ax = plt.subplots(figsize=(10, 7))
importance.head(20).sort_values().plot.barh(ax=ax)
ax.set_title("XGBoost Feature Importance — Top 20")
ax.set_xlabel("Importance Score")
plt.tight_layout()
plt.savefig("feature_importance.png", dpi=120)
plt.close()
print("Saved feature_importance.png")

Top 20 features:
Total_Dependents                         0.124701
Broker_Agency_Type_Urban_Boutique        0.119457
Broker_Agency_Type_National_Corporate    0.101011
Minor_Dependents                         0.068286
Income_Bracket                           0.062473
Has_Broker                               0.050862
Adult_Dependents                         0.044284
Policy_Start_Year                        0.038655
Child_Dependents                         0.037519
Acquisition_Channel_Direct_Website       0.034685
Broker_ID                                0.027283
Acquisition_Channel_Affiliate_Group      0.015240
Acquisition_Channel_Corporate_Partner    0.015079
Deductible_Ord                           0.015042
Month_Sin                                0.014764
Employment_Status_Self_Employed          0.014628
Acquisition_Channel_Aggregator_Site      0.014125
Region_Enc                               0.012329
Employment_Status_Employed_FullTime      0.011942
Log_Income                       

## 11. Generate Submission

In [23]:
y_test_pred   = xgb_model.predict(X_test_final.values)
y_test_labels = le.inverse_transform(y_test_pred)

submission = pd.DataFrame({
    "User_ID": test_ids,
    TARGET:    y_test_labels
})
submission.to_csv("xgb_submission.csv", index=False)
print(f"Submission saved — {len(submission):,} rows")
print(submission.head(10).to_string(index=False))

Submission saved — 15,218 rows
   User_ID  Purchased_Coverage_Bundle
USR_060868                          0
USR_060869                          2
USR_060870                          2
USR_060871                          4
USR_060872                          2
USR_060873                          4
USR_060874                          4
USR_060875                          6
USR_060876                          3
USR_060877                          2


## 12. Save Full Artifact

In [24]:
artifact = {
    "model":           xgb_model,
    "feature_names":   feature_names,
    "label_encoder":   le,
    "region_map":      region_map,
    "global_mean":     global_mean,
    "medians":         medians.to_dict(),
    "cat_cols":        cat_cols,
    "num_classes":     NUM_CLASSES,
}
joblib.dump(artifact, "model.joblib")
print("Artifact saved → model.joblib")

Artifact saved → model.joblib
